In [1]:
import json
from glob import glob
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import ColumnDataSource, Whisker
from bokeh.models import Range1d
from bokeh.sampledata.autompg import autompg as df

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def concatenate_results(results, new_res):
    if not 'total' in results['events']:
        results['events']['total'] = {}
    for event, dict_event in new_res.items():
        if not event in results['events']:
            results['events'][event] = {}
        for key, value in dict_event['fixations'].items():
            if not key in results['events']['total']:
                results['events']['total'][key] = {}
                results['events']['total'][key]['data'] = []
            if not key in results['events'][event]:
                results['events'][event][key] = {}
                results['events'][event][key]['data'] = []
            tmp = 0
            for fix in value:
                tmp = tmp +  fix['ratio']
            results['events'][event][key]['data'].append(tmp)
            results['events']['total'][key]['data'].append(tmp)
    return results

In [4]:
def style(p):
    # Title 
    p.title.align = 'center'
    p.y_range = Range1d(-0.2, 1.0)
    p.xaxis.ticker = [1, 2, 3]
    return p

In [5]:
def create_grid(disp_event, disp_fix):
    figures = []
    for e in disp_event:
        for f in disp_fix:      
                s = figure(plot_width=400, plot_height=200, title=e + "_" + f)
                lower = []
                upper = []
                base = [1,2,3]
                for sess in keys[1:]:
                    if f in results[sess]['events'][e].keys():
                        lower.append(results[sess]['events'][e][f]['mean'] - results[sess]['events'][e][f]['std'])
                        upper.append(results[sess]['events'][e][f]['mean'] + results[sess]['events'][e][f]['std'])
                    else:
                        lower.append(0)
                        upper.append(0)

                source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
                s.add_layout(
                    Whisker(source=source_error, base="base", upper="upper", lower="lower")
                )

                for i, sess in enumerate(keys[1:]):
                    if f in results[sess]['events'][e].keys():
                        y = results[sess]['events'][e][f]['data']
                        color = colors[i % len(colors)]
                        s.circle(x=base[i], y=y, color=color)
                s = style(s)
                figures.append(s)
    return gridplot(figures, ncols=2)

In [6]:
files = glob('../json/*.json')

In [7]:
data = {}
for f in files:
    with open(f) as datafile:
        d = json.load(datafile)
    tmp = f.split('/')[-1][:-5].split('_')
    if not tmp[0] in data:
        data[tmp[0]] = {}
    data[tmp[0]][tmp[1]] = d

In [8]:
results = {}
keys = ['00', '11', '12', '13', ]
for k in keys:
    results[k] = {}
    results[k]['nb_subjects'] = 0
    results[k]['events'] = {}

In [9]:
results = {}
keys = ['00', '11', '12', '13']
for k in keys:
    results[k] = {}
    results[k]['nb_subjects'] = 0
    results[k]['events'] = {}
for key, value in data.items():
    for sess in keys:
        if sess in value.keys():
            results[sess]['nb_subjects'] = results[sess]['nb_subjects'] + 1
            results[sess] = concatenate_results(results[sess], value[sess])
            
for sess, sess_dict in results.items():
    for event, event_dict in sess_dict['events'].items():
        for fix, value in event_dict.items():
            results[sess]['events'][event][fix]['mean'] = np.mean(value['data'])
            results[sess]['events'][event][fix]['std'] = np.std(value['data'])

In [10]:
colors = ["red", "olive", "darkred"]
disp_fix = ['LHand', 'RHand', 'Head', 'VisualServoing', 'Object', 'Direction']

In [11]:
disp_event = ['MoveUp']
show(create_grid(disp_event, disp_fix))

In [12]:
disp_event = ['MoveDown']
show(create_grid(disp_event, disp_fix))

In [13]:
disp_event = ['Move']
show(create_grid(disp_event, disp_fix))

In [14]:
disp_event = ['Regrasp']
show(create_grid(disp_event, disp_fix))

In [15]:
disp_event = ['Place']
show(create_grid(disp_event, disp_fix))

In [16]:
disp_event = ['total']
show(create_grid(disp_event, disp_fix))